# Demonstração de Overfitting vs Dropout Regularization

Este notebook demonstra o efeito do overfitting em redes neurais profundas e como a técnica de **Dropout** pode mitigar esse problema.

## Objetivo
Treinar duas redes neurais idênticas no dataset MNIST:
1. **Sem Dropout**: Propenso a overfitting
2. **Com Dropout**: Regularização para melhor generalização

## Configuração
- Dataset: MNIST (subset de 1000 amostras de treino)
- Arquitetura: 9 camadas densas (2048→1024→1024→512→512→256→256→128→64→10)
- Dropout rate: 0.3
- Épocas: 100
- Batch size: 32

## 1. Importação de Bibliotecas

In [ ]:


import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")
print(f"Devices: {tf.config.list_physical_devices()}")

## 2. Carregamento e Preparação dos Dados

Carregamos o dataset MNIST e criamos um **subset pequeno** de apenas 1000 amostras de treino para facilitar o overfitting.

In [ ]:
# Carregar MNIST
(x_train_full, y_train_full), (x_test, y_test) = mnist.load_data()

# Normalizar para [0, 1]
x_train_full = x_train_full.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten: 28x28 -> 784
x_train_full = x_train_full.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# Criar subset pequeno para facilitar overfitting
subset_size = 1000
x_train_subset, _, y_train_subset, _ = train_test_split(
    x_train_full[:subset_size], 
    y_train_full[:subset_size], 
    test_size=0.0, 
    random_state=42
)

print(f"Training subset: {x_train_subset.shape}, {y_train_subset.shape}")
print(f"Test set: {x_test.shape}, {y_test.shape}")

## 3. Definição dos Modelos

Criamos duas arquiteturas **idênticas**:
- **Modelo 1**: Sem Dropout (propenso a overfitting)
- **Modelo 2**: Com Dropout (rate=0.2) para regularização

Ambos usam 9 camadas densas com ReLU activation.

In [ ]:
def create_model_without_dropout():
    """Modelo sem Dropout - propenso a overfitting"""
    model = Sequential([
        Input(shape=(784,)),
        Dense(2048, activation='relu'),
        Dense(1024, activation='relu'),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=0.01),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def create_model_with_dropout():
    """Modelo com Dropout - regularização para melhor generalização"""
    model = Sequential([
        Input(shape=(784,)),
        Dense(2048, activation='relu'),
        Dropout(0.2),
        Dense(1024, activation='relu'),
        Dropout(0.2),
        Dense(1024, activation='relu'),
        Dropout(0.2),
        Dense(512, activation='relu'),
        Dropout(0.2),
        Dense(512, activation='relu'),
        Dropout(0.2),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=0.01),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("Modelos definidos com sucesso!")

## 4. Treinamento dos Modelos

Treinamos ambos os modelos por 50 épocas e coletamos as métricas de treino e teste.

In [ ]:
# Hiperparâmetros
epochs = 50
batch_size = 128

print("=" * 60)
print("EXPERIMENTO 1: Modelo SEM Dropout")
print("=" * 60)

model_no_dropout = create_model_without_dropout()
history_no_dropout = model_no_dropout.fit(
    x_train_subset, y_train_subset,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    verbose=1
)

print("\n" + "=" * 60)
print("EXPERIMENTO 2: Modelo COM Dropout")
print("=" * 60)

model_with_dropout = create_model_with_dropout()
history_with_dropout = model_with_dropout.fit(
    x_train_subset, y_train_subset,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    verbose=1
)

print("\nTreinamento concluído!")

## 5. Visualização dos Resultados

Criamos 5 gráficos para analisar o desempenho:
1. Loss do modelo sem dropout
2. Acurácia do modelo sem dropout
3. Loss do modelo com dropout
4. Acurácia do modelo com dropout
5. Comparação final das métricas

In [ ]:
# Extrair métricas finais
train_acc_no_dropout = history_no_dropout.history['accuracy'][-1]
test_acc_no_dropout = history_no_dropout.history['val_accuracy'][-1]
train_loss_no_dropout = history_no_dropout.history['loss'][-1]
test_loss_no_dropout = history_no_dropout.history['val_loss'][-1]

train_acc_with_dropout = history_with_dropout.history['accuracy'][-1]
test_acc_with_dropout = history_with_dropout.history['val_accuracy'][-1]
train_loss_with_dropout = history_with_dropout.history['loss'][-1]
test_loss_with_dropout = history_with_dropout.history['val_loss'][-1]

# Criar visualizações
fig = plt.figure(figsize=(20, 12))

# 1. Loss sem dropout
ax1 = plt.subplot(2, 3, 1)
ax1.plot(history_no_dropout.history['loss'], label='Treino', linewidth=2)
ax1.plot(history_no_dropout.history['val_loss'], label='Teste', linewidth=2)
ax1.set_xlabel('Época', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Experimento 1: Loss (Sem Dropout)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# 2. Accuracy sem dropout
ax2 = plt.subplot(2, 3, 2)
ax2.plot(history_no_dropout.history['accuracy'], label='Treino', linewidth=2)
ax2.plot(history_no_dropout.history['val_accuracy'], label='Teste', linewidth=2)
ax2.set_xlabel('Época', fontsize=12)
ax2.set_ylabel('Acurácia', fontsize=12)
ax2.set_title('Experimento 1: Acurácia (Sem Dropout)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

# 3. Loss com dropout
ax3 = plt.subplot(2, 3, 4)
ax3.plot(history_with_dropout.history['loss'], label='Treino', linewidth=2)
ax3.plot(history_with_dropout.history['val_loss'], label='Teste', linewidth=2)
ax3.set_xlabel('Época', fontsize=12)
ax3.set_ylabel('Loss', fontsize=12)
ax3.set_title('Experimento 2: Loss (Com Dropout)', fontsize=14, fontweight='bold')
ax3.legend(fontsize=11)
ax3.grid(True, alpha=0.3)

# 4. Accuracy com dropout
ax4 = plt.subplot(2, 3, 5)
ax4.plot(history_with_dropout.history['accuracy'], label='Treino', linewidth=2)
ax4.plot(history_with_dropout.history['val_accuracy'], label='Teste', linewidth=2)
ax4.set_xlabel('Época', fontsize=12)
ax4.set_ylabel('Acurácia', fontsize=12)
ax4.set_title('Experimento 2: Acurácia (Com Dropout)', fontsize=14, fontweight='bold')
ax4.legend(fontsize=11)
ax4.grid(True, alpha=0.3)

# 5. Comparação final
ax5 = plt.subplot(1, 3, 3)
x_pos = np.arange(4)
width = 0.35

bars_no_dropout = [train_acc_no_dropout, test_acc_no_dropout, 
                   train_loss_no_dropout, test_loss_no_dropout]
bars_with_dropout = [train_acc_with_dropout, test_acc_with_dropout, 
                     train_loss_with_dropout, test_loss_with_dropout]

bar1 = ax5.bar(x_pos - width/2, bars_no_dropout, width, 
               label='Sem Dropout', alpha=0.8, color='#e74c3c')
bar2 = ax5.bar(x_pos + width/2, bars_with_dropout, width, 
               label='Com Dropout', alpha=0.8, color='#3498db')

ax5.set_xlabel('Métrica', fontsize=12)
ax5.set_ylabel('Valor', fontsize=12)
ax5.set_title('Comparação Final das Métricas', fontsize=14, fontweight='bold')
ax5.set_xticks(x_pos)
ax5.set_xticklabels(['Acc Treino', 'Acc Teste', 'Loss Treino', 'Loss Teste'], 
                     rotation=15, ha='right')
ax5.legend(fontsize=11)
ax5.grid(True, alpha=0.3, axis='y')

# Adicionar valores nas barras
for bars in [bar1, bar2]:
    for bar in bars:
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('overfitting_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "=" * 60)
print("RESULTADOS FINAIS")
print("=" * 60)
print(f"\nSEM DROPOUT:")
print(f"  Acurácia Treino: {train_acc_no_dropout:.3f}")
print(f"  Acurácia Teste:  {test_acc_no_dropout:.3f}")
print(f"  Gap (Overfitting): {train_acc_no_dropout - test_acc_no_dropout:.3f}")
print(f"  Loss Treino: {train_loss_no_dropout:.3f}")
print(f"  Loss Teste:  {test_loss_no_dropout:.3f}")

print(f"\nCOM DROPOUT:")
print(f"  Acurácia Treino: {train_acc_with_dropout:.3f}")
print(f"  Acurácia Teste:  {test_acc_with_dropout:.3f}")
print(f"  Gap (Overfitting): {train_acc_with_dropout - test_acc_with_dropout:.3f}")
print(f"  Loss Treino: {train_loss_with_dropout:.3f}")
print(f"  Loss Teste:  {test_loss_with_dropout:.3f}")

print("\n" + "=" * 60)
print("ANÁLISE")
print("=" * 60)
print(f"Redução no Gap de Acurácia: "
      f"{((train_acc_no_dropout - test_acc_no_dropout) - (train_acc_with_dropout - test_acc_with_dropout)):.3f}")
print(f"Melhoria na Acurácia de Teste: "
      f"{(test_acc_with_dropout - test_acc_no_dropout):.3f}")
print("=" * 60)

## Conclusões

Este experimento demonstra claramente o fenômeno de **overfitting** e como o **Dropout** pode mitigá-lo:

### Observações Principais:
1. **Overfitting Sem Dropout**: O modelo sem dropout alcança alta acurácia no conjunto de treino, mas desempenho significativamente inferior no conjunto de teste (gap grande)
2. **Regularização com Dropout**: O modelo com dropout apresenta menor gap entre treino e teste, indicando melhor generalização
3. **Trade-off**: O dropout pode reduzir ligeiramente a acurácia de treino, mas melhora a capacidade de generalização

### Métricas-Chave:
- **Gap de Acurácia**: Diferença entre acurácia de treino e teste (quanto menor, melhor)
- **Acurácia de Teste**: Métrica mais importante para avaliar generalização
- **Loss**: Comportamento da função de perda ao longo das épocas

### Próximos Passos:
- Experimentar diferentes dropout rates (0.1, 0.3, 0.5)
- Adicionar outras técnicas de regularização (L1, L2, Batch Normalization)